In [ ]:
import pandas as pd
gpt_tag=pd.read_csv("/content/drive/MyDrive/colab_github/sooljari-crawling/gptoutput.csv")


In [122]:
cnt=0
errlst = []
for item in gpt_tag['gptoutput']:
  if(type(item)==type("str")):
    newstr=item.replace(" ","").replace("\n","").replace(".","")
    #gpt_tag=gpt_tag.replace({'gptoutput' : item}, newstr)
    if(newstr[0]!='#'):
      print(cnt)
      errlst.append(cnt)
  else:
    print(cnt)
    errlst.append(cnt)
  cnt=cnt+1

In [ ]:
#print(str(errlst[0])+" "+gpt_tag['텍스트 내용'][errlst[0]]+".")

In [112]:
#gpt_tag.loc[errlst[0], 'gptoutput'] = "#가야양조장, #기다림과정성, #발효, #숙성과정, #2차숙성, #풍부한맛과향, #전통농담금법, #어머니의정성, #평균66일, #저온숙성"
#del errlst[0]

In [124]:
gpt_tag.to_csv('gptoutput_refined.csv')